Chetana Iyer 
Github username: chetiyer

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create an instance of the network and define the loss and optimizer
net = Net()
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())

# Convert the data to PyTorch tensors
X = torch.from_numpy(np.arange(0,31).reshape(-1,1).astype(np.float32))
Y = torch.from_numpy(np.array([30, 35, 33, 32, 34, 37, 39, 38, 36, 36, 37, 39, 42, 45, 45, 41,
                                40, 39, 42, 44, 47, 49, 50, 49, 46, 48, 50, 53, 55, 54, 53]).astype(np.float32))

Training loss: tensor(1401.8444)
Test loss: tensor(2446.0713)


/Users/chetanaiyer/miniconda/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/chetanaiyer/miniconda/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/chetanaiyer/miniconda/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([11])) that is different to the input size (torch.Size([11, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure 

In [4]:
# (ii) Using the first 20 data points as training data, fit the neural network
X_train = X[:20]
Y_train = Y[:20]

for epoch in range(1000):
    optimizer.zero_grad()
    output = net(X_train)
    loss = criterion(output, Y_train)
    loss.backward()
    optimizer.step()

# Compute the least-square error for each of these over the training points
with torch.no_grad():
    train_loss = criterion(net(X_train), Y_train)
    print('Training loss:', train_loss)

# Compute the least square error of these models on the test data which are the remaining 10 data points
X_test = X[20:]
Y_test = Y[20:]
with torch.no_grad():
    test_loss = criterion(net(X_test), Y_test)
    print('Test loss:', test_loss)


# (iii) Repeat (iii) but use the first 10 and last 10 data points as training data
X_train2 = torch.cat((X[:10], X[-10:]), dim=0)
Y_train2 = torch.cat((Y[:10], Y[-10:]), dim=0)

for epoch in range(1000):
    optimizer.zero_grad()
    output = net(X_train2)
    loss = criterion(output, Y_train2)
    loss.backward()
    optimizer.step()

# Compute the least-square error for each of these over the training points
with torch.no_grad():
    train_loss2 = criterion(net(X_train2), Y_train2)
    print('Training loss (ii):', train_loss2)

# Fit the model to the test data (which are the 10 held out middle data points)
X_test2 = X[10:20]
Y_test2 = Y[10:20]
with torch.no_grad():
    test_loss2 = criterion(net(X_test2), Y_test2)
    print('Test loss (ii):', test_loss2)


Training loss: tensor(1333.6720)
Test loss: tensor(2355.3342)
Training loss (ii): tensor(1661.1974)
Test loss (ii): tensor(1485.2930)


/Users/chetanaiyer/miniconda/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Homework 1 looked at fitting data to a line,parabola and 19th degree polynomial - for the first splitting 20 points of the data each evenly into training and test, the errors were respectively for

training: 
line fitting - 100.59
parabola - 90.35
19th degree polynomial - 0.016

and test  
line fitting - 118.28
parabola - 816.33
19th degree polynomial - 9.014

on which set, the neural networks performed worse with a training loss of 1333.67, and a test loss of 2355.33

For the second division of the data, 
the respective values were 

training: 
line fitting - 68.57
parabola - 68.51
19th degree polynomial - 0.536

and test  
line fitting - 86.64
parabola - 84.70
19th degree polynomial - 2576550.371

The training and test loss for the neural network was 1661.19 and 1485.2930 respectively. It performed better than the 19th degree polynomial on the testing data, but worse compared to both line fitting and parabola fitting 

# II Now train a feedforward neural network on the MNIST data set. You will start by performing the following analysis:
### (i) Compute the first 20 PCA modes of the digit images.


In [6]:
import numpy as np
from sklearn.decomposition import PCA
from torchvision import datasets, transforms

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Flatten the images
train_images = mnist_trainset.data.reshape(mnist_trainset.data.shape[0], -1)
test_images = mnist_testset.data.reshape(mnist_testset.data.shape[0], -1)

# Perform PCA on the training images
pca = PCA(n_components=20)
train_images_pca = pca.fit_transform(train_images)

# Use the PCA model to transform the test images
test_images_pca = pca.transform(test_images)


100%|███████████████████████████| 9912422/9912422 [00:00<00:00, 50103714.11it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|████████████████████████████████| 28881/28881 [00:00<00:00, 4439156.18it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|████████████████████████████| 1648877/1648877 [00:00<00:00, 9676121.04it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 3613529.74it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



### (ii) Build a feed-forward neural network to classify the digits. 

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the network and define the loss and optimizer
input_size = 20 # size of PCA output
hidden_size = 50
output_size = 10 # number of classes
net = Net(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

# Convert the data to PyTorch tensors
train_labels = mnist_trainset.targets
test_labels = mnist_testset.targets
X_train = torch.from_numpy(train_images_pca.astype(np.float32))
y_train = torch.from_numpy(train_labels.numpy())
X_test = torch.from_numpy(test_images_pca.astype(np.float32))
y_test = torch.from_numpy(test_labels.numpy())

# Train the network
num_epochs = 10
batch_size = 32
for epoch in range(num_epochs):
    for i in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        output = net(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()

# Evaluate the network
net.eval()
with torch.no_grad():
    train_preds = torch.argmax(net(X_train), axis=1)
    test_preds = torch.argmax(net(X_test), axis=1)
    train_acc = torch.sum(train_preds == y_train).item() / len(y_train)
    test_acc = torch.sum(test_preds == y_test).item() / len(y_test)

print("Training accuracy:", train_acc)
print("Test accuracy:", test_acc)


Training accuracy: 0.9094666666666666
Test accuracy: 0.9106


## Compare the results of the neural network against LSTM, SVM (support vector machines) and decision tree classifiers.

In [21]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Flatten the images
train_images = mnist_trainset.data.reshape(mnist_trainset.data.shape[0], -1)
test_images = mnist_testset.data.reshape(mnist_testset.data.shape[0], -1)

# Perform PCA on the training images
pca = PCA(n_components=20)
train_images_pca = pca.fit_transform(train_images)

# Use the PCA model to transform the test images
test_images_pca = pca.transform(test_images)

# Load labels
train_labels = mnist_trainset.targets.numpy()
test_labels = mnist_testset.targets.numpy()

# Convert the data to PyTorch tensors
train_images_pca_tensor = torch.from_numpy(train_images_pca.astype(np.float32))
train_labels_tensor = torch.from_numpy(train_labels)
test_images_pca_tensor = torch.from_numpy(test_images_pca.astype(np.float32))
test_labels_tensor = torch.from_numpy(test_labels)

# Create PyTorch datasets
train_dataset = TensorDataset(train_images_pca_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_pca_tensor, test_labels_tensor)

# Set up the DataLoader objects
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Define the LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Set up the model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 20
hidden_size = 128
num_layers = 2
num_classes = 10
learning_rate = 0.001

model_lstm = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=learning_rate)

# Train the LSTM model
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = 0.0
    total = 0
    correct = 0
    for images, labels in train_loader:
        images = images.view(-1, 1, input_size).to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model_lstm(images)
        loss = criterion(outputs, labels)
        loss.backward()
       


In [ ]:
# Evaluate the LSTM model using scikit-learn accuracy score
with torch.no_grad():
    predicted_labels = []
    true_labels = []
    for images, labels in test_loader:
        images = images.view(-1, 1, input_size).to(device)
        outputs = model_lstm(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, predicted_labels)
print(f'Test accuracy (LSTM): {accuracy:.4f}')


The Test accuracy (LSTM) of 0.0999 is significantly lower than the Training accuracy of 0.9094666666666666 for the LSTM model, and also much lower than the Test accuracy of 0.9106 for the feedforward neural network on the same dataset.

This indicates that the LSTM model is overfitting to the training data and is not generalizing well to the test data. This could be due to various reasons such as suboptimal hyperparameters, insufficient training data, or insufficient regularization.

It may be worth experimenting with different hyperparameters or regularization techniques to see if the performance of the LSTM model can be improved.

# Evaluating against SVM and decision tree classifiers

In the last homework assignment, we calculated the accuracy of SVM and decision tree classifiers in overall classification. The overall accuracy for classification for the SVM and Decision tree methods were 0.97628 and 0.8666. 

From this homework, we identified that the classification accuracy for the feed-forward neural network was 0.9106, and 0.0999 for the LSTM model. 

From this we can note the order of performance in classifying MNIST digits,  as 1) SVM, 2) FFNN 3) Decision tree and lastly LSTM 